# 02_post-training
### Only quantization
 * [Dynamic range quantization](#Dynamic-range-quantization)
 * [Float16 quantization](#Float16-quantization)
 * [Full integer quantization](#Full-integer-quantization)
     - Integer with float fallback
     - Integer only

In [1]:
import numpy as np
import tensorflow as tf

### Dynamic range quantization

In [2]:
# path to the SavedModel directory
saved_model_dir = 'model/mobilenet_saved_model'

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

# Save the model.
with open('tflite_model/mobilenet_quant.tflite', 'wb') as f:
    f.write(tflite_quant_model)

### Float16 quantization

In [3]:
# path to the SavedModel directory
saved_model_dir = 'model/mobilenet_saved_model'

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quant_model = converter.convert()

# Save the model.
with open('tflite_model/mobilenet_quant_fp16.tflite', 'wb') as f:
    f.write(tflite_quant_model)

### Full integer quantization

In [4]:
# Define data generator
x_train = tf.random.uniform([1000, 224, 224, 3])
def representative_dataset_gen():
    for img in x_train:
        img = np.expand_dims(img, axis=0)
        yield [img]

#### Integer with float fallback

In [5]:
# path to the SavedModel directory
saved_model_dir = 'model/mobilenet_saved_model'

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
tflite_quant_model = converter.convert()

# Save the model.
with open('tflite_model/mobilenet_quant_int8.tflite', 'wb') as f:
    f.write(tflite_quant_model)

#### Integer only

In [6]:
# path to the SavedModel directory
saved_model_dir = 'model/mobilenet_saved_model'

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
converter.representative_dataset = representative_dataset_gen
tflite_quant_model = converter.convert()

# Save the model.
with open('tflite_model/mobilenet_quant_int8_only.tflite', 'wb') as f:
    f.write(tflite_quant_model)